In [237]:
from bs4 import BeautifulSoup
import requests
import csv

with open('NeighborhoodToronto.html') as html_file:
    soup = BeautifulSoup(html_file,'lxml')

In [238]:
#Web Scraping
csv_file = open('Toronto_scraping.csv','w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postal Code','Borough','Neighborhood'])

table = soup.find('table',style = 'width:100%; border-collapse:collapse; border:1px solid #ccc;')

for info in table.find_all('td'):
    
    Postalcode = info.find('b').text
    print(Postalcode)
    
    a = info.find('span',style ='font-size:80%;').text
    Borough = a.split('(')[0]
    print(Borough)
    
    cpt = 0
    Neighborhood =""

    for a in info.find_all('a'):
          
        if cpt==1:
            Neighborhood =""
            try:
            #Neighborhood = a.split('(')[1]
            #Neighborhood = Neighborhood.split(')')[0]
                test = a.text
                Neighborhood =  test 
        
            except Exception as e:
                Neighborhood = None
        elif cpt>1:
            
            try:
            #Neighborhood = a.split('(')[1]
            #Neighborhood = Neighborhood.split(')')[0]
                test = a.text
                Neighborhood =  Neighborhood +","+test
        
            except Exception as e:
                Neighborhood = None
        
            
        cpt=cpt+1
    print(Neighborhood)    
    cpt = 0
    print()
    
    csv_writer.writerow([Postalcode,Borough,Neighborhood])

csv_file.close()

M1A
Not assigned


M2A
Not assigned


M3A
North York
Parkwoods

M4A
North York
Victoria Village

M5A
Downtown Toronto
Regent Park,Harbourfront

M6A
North York
Lawrence Manor,Lawrence Heights

M7A
Queen's Park / Ontario Provincial Government
Ontario Provincial Government

M8A
Not assigned


M9A
Etobicoke
Islington Avenue

M1B
Scarborough
Malvern,Rouge

M2B
Not assigned


M3B
North York
Don Mills

M4B
East York
Parkview Hill,Woodbine Gardens

M5B
Downtown Toronto
Garden District,Ryerson

M6B
North York


M7B
Not assigned


M8B
Not assigned


M9B
Etobicoke
West Deane Park,Princess Gardens,Martin Grove,Islington

M1C
Scarborough
Rouge Hill,Port Union,Highland Creek

M2C
Not assigned


M3C
North York
Don Mills,Flemingdon Park

M4C
East York
Woodbine Heights

M5C
Downtown Toronto
St. James Town

M6C
York
Humewood-Cedarvale

M7C
Not assigned


M8C
Not assigned


M9C
Etobicoke
Markland Wood

M1E
Scarborough
Guildwood,Morningside,West Hill

M2E
Not assigned


M3E
Not assigned


M4E
East Toronto

In [239]:
#Download dataframe
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
df = pd.read_csv("Toronto_scraping.csv")
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"


In [240]:
#Remove Not assigned line from the dataframe
df = df.loc[df['Borough']!="Not assigned",:]
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government


In [241]:
#neighborhood same as borough when borugh is not assigned
df['Neighborhood'] = df['Neighborhood'].replace(np.nan, df['Borough']) 
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government


In [242]:
df.shape

(103, 3)

In [244]:
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")
df_coord.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [251]:
neighborhoods = pd.merge(left=df, right=df_coord, left_on='Postal Code', right_on='Postal Code')

In [252]:
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Old Mill",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,East TorontoBusiness reply mail Processing Cen...,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill,Sunnylea,Humber Bay,Mimico,The Queensway",43.636258,-79.498509


In [265]:
#recover only data from Toronto
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,East Toronto,43.676357,-79.293031
4,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306


In [266]:
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [267]:
import folium
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto



In [268]:
CLIENT_ID = 'WVUAL3VW1MOLE4LWOTBOHT4LYLQPU3XGMPGCXQML0OQQAJHA' # your Foursquare ID
CLIENT_SECRET = 'BSPTJGUFKD5DHMJ0JFCZIH1M2FC0A0N00CZ34VUKEJEG1SVZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WVUAL3VW1MOLE4LWOTBOHT4LYLQPU3XGMPGCXQML0OQQAJHA
CLIENT_SECRET:BSPTJGUFKD5DHMJ0JFCZIH1M2FC0A0N00CZ34VUKEJEG1SVZ


In [270]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park,Harbourfront
Garden District,Ryerson
St. James Town
East Toronto
Downtown Toronto
Bay Street
Downtown Toronto
Richmond,King
Dovercourt Village
The Danforth
Harbourfront,Union Station,Toronto Islands
Trinity
Riverdale
Toronto Dominion Centre,Design Exchange
Parkdale Village,Exhibition Place
The Beaches
Commerce Court,Victoria Hotel
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
The Junction
Central Toronto
North Midtown,Yorkville
Roncesvalles
Central Toronto
University of Toronto
Swansea
Summerhill
Kensington Market,Chinatown,Grange Park
Rathnelly,South Hill,Forest Hill,Deer Park
CN Tower,King and Spadina,Railway Lands,Harbourfront,South Niagara,Island airport
Rosedale
Downtown TorontoStn A PO Boxes25 The Esplanade
St. James Town,Cabbagetown
First Canadian Place,Underground city
Church and Wellesley
East TorontoBusiness reply mail Processing Centre969 Eastern


In [271]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,83,83,83,83,83,83
"CN Tower,King and Spadina,Railway Lands,Harbourfront,South Niagara,Island airport",16,16,16,16,16,16
Central Toronto,74,74,74,74,74,74
Church and Wellesley,87,87,87,87,87,87
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Dovercourt Village,19,19,19,19,19,19
Downtown Toronto,74,74,74,74,74,74
Downtown TorontoStn A PO Boxes25 The Esplanade,95,95,95,95,95,95
East Toronto,45,45,45,45,45,45


In [272]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [273]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [274]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bay Street,0.012048,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012048,...,0.000000,0.00,0.000000,0.012048,0.000000,0.000000,0.012048,0.000000,0.000000,0.00
1,"CN Tower,King and Spadina,Railway Lands,Harbou...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Toronto,0.013514,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.013514,...,0.013514,0.00,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Church and Wellesley,0.011494,0.011494,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011494,...,0.000000,0.00,0.000000,0.000000,0.000000,0.011494,0.000000,0.011494,0.011494,0.00
4,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
5,Dovercourt Village,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Downtown Toronto,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.010526,...,0.000000,0.00,0.000000,0.010526,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,East Toronto,0.022222,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.044444,...,0.022222,0.00,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.00
9,East TorontoBusiness reply mail Processing Cen...,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [275]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [277]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Salad Place,Café,Juice Bar,Thai Restaurant
1,"CN Tower,King and Spadina,Railway Lands,Harbou...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
2,Central Toronto,Park,Sandwich Place,Dessert Shop,Coffee Shop,Café,Clothing Store,Gym,Pizza Place,Sushi Restaurant,Bus Line
3,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Bubble Tea Shop,Hotel,Café,Mediterranean Restaurant,Pub
4,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Japanese Restaurant,Gastropub,Deli / Bodega,Seafood Restaurant


In [279]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 0, 0, 3, 0, 0, 3, 3], dtype=int32)

In [280]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Café,Theater,Mexican Restaurant,Event Space,Farmers Market,Breakfast Spot
1,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Diner,Bookstore,Electronics Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Clothing Store,Diner,Beer Bar,Cosmetics Shop,Breakfast Spot
3,M4E,East Toronto,East Toronto,43.676357,-79.293031,3,Café,Coffee Shop,Bakery,Italian Restaurant,Brewery,American Restaurant,Yoga Studio,Clothing Store,Seafood Restaurant,Bookstore
4,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306,0,Coffee Shop,Café,Grocery Store,Park,Restaurant,Bakery,Beer Bar,Cheese Shop,Cocktail Bar,Nightclub


In [282]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters